<font size=5>**Part IV Final Model & Outliers**</font> <br/>

<font size=4>**3.1 Final model description**</font> <br/>
We choose weight and abdomen as predictiors for body fat as our final model.

In [8]:
lm.reg=lm(BODYFAT~WEIGHT+ABDOMEN,fat.rm)

Our final model is 
$$\text{BodyFat} = -42.95790 -0.11994 \cdot \text{WEIGHT} + 0.90152 \cdot \text{ABDOMEN} $$
The coefficients are significant and the adjusted R-squared is 0.7148. 
When weight increases one additinal unit(lbs), the bodyfat will decrease 0.11994%. When abdomen increases one additional unit(cm), the bodyfat will increase 0.90152%.

<font size=4>**3.2 Advantages and disadvantages**</font> <br/>
**Advantages**: The model is very simple since there are only two variables and it is convenient for users to gain their weight and abdomen data.<br\>
**Disadvantages**: Since there are only two variables, we sacrificed some accuracy of this model. Meanwhile, the data of users' abdomen may not be as convenient as variables like height and weight to obtain.

<font size=4>**3.3 Model Diagnose & Outliers**</font> <br/>
<font size=3>**3.3.1 Assumption diagnose and outlier detection**</font> <br/>
To futherly estimate our model, we draw plots to check the assumptions and outliers for this model. As we can see from the residuals vs fitted values plot, the assumpiton of equal variance and linearity have been satisfied. Besides, From the qqplot, we can see that the normality assumption has been satisfied.We also drew standard / studentized residuals vs fitted values plots and find that there is no significant outlier.

<font size=3>**3.3.2 Influential points detection** </font> <br/>
To determine some influential points, we draw Leverage plot, Diffits plot, Cook's distance plot and Debates plot. Aside from Leverage plot, the other three plots does not find influential points. From the Leverage plot, we find 8 influential points(9,12,35,36,152,216,242th). However, there is no enough evidence showing that they are outliers even though they are infulential.
